# Setting up a Schema in Spark

### Introduction

Now in the last lesson, we were able to explore the dataset of civil complaints, but we did so with all of our data set to strings.  This will reduce our ability to explore, sort, and perform aggregations on our dataset.  In this lesson, we'll see how we can set the proper schema on a Spark dataframe.  

Let's get started.

### Getting Set Up (For Google Colab)

> If we are running this on google colab, we can run the following to eventually interact with our Spark UI.

* Begin by installing some pip packages and the java development kit.

In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

* Then set the java environmental variable

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

* Then connect to a SparkSession, setting the spark ui port to `4050`.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("civComplaints") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()

* Then we need to install ngrok which will allow us to place our local spark ui on the web.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')

* And finally we get a link our Spark UI

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Loading our Data

Then, let's read in the our csv data.

In [ ]:
import pandas as pd

df = pd.read_csv('s3://jigsaw-labs/civ_complaints.csv').astype(str)

There are a lot of columns to this dataset, so let's set `vertical = True`. 

And then again, we'll create a spark dataframe.

In [ ]:
complaints_df = spark.createDataFrame(df)

And then let's take a look at the schema.

In [ ]:
complaints_df.printSchema()

root
 |-- Extract Run Date: string (nullable = true)
 |-- Randomized Id: string (nullable = true)
 |-- CCRB Received Year: string (nullable = true)
 |-- Days Between Incident Date and Received Date: string (nullable = true)
 |-- Case Type: string (nullable = true)
 |-- Complaint Received Place: string (nullable = true)
 |-- Complaint Received Mode: string (nullable = true)
 |-- Borough Of Incident: string (nullable = true)
 |-- Patrol Borough Of Incident: string (nullable = true)
 |-- Reason For Initial Contact: string (nullable = true)



So we can see that all of the columns are currently set to datatype of string.  But if we set columns like `Extract Run Date` to datetimes, then we can perform calculations like selecting the month or day of week from those dates.  And even more columns should be numeric.  Ok, so let's see how we can set our columns to the correct datatypes.

### Exploring withColumn

It turns out that we can use the `withColumn` method to change our data type.  Now the `withColumn` is generally used to derive new values from an existing column.  For example, if we look at the values in our `Randomized Id` column.

In [ ]:
complaints_df.select('Randomized Id').show(3)

+-------------+
|Randomized Id|
+-------------+
|            1|
|            2|
|            3|
+-------------+
only showing top 3 rows



And then we can used `withColumn` to increment each id by two like so:

In [ ]:
updated_complaints_df = complaints_df.withColumn("id_plus_two", 
                                                 col("Randomized Id") + 2)

In [ ]:
updated_complaints_df.select(["Randomized Id", "id_plus_two"]).show(5)

+-------------+-----------+
|Randomized Id|id_plus_two|
+-------------+-----------+
|            1|        3.0|
|            2|        4.0|
|            3|        5.0|
|            4|        6.0|
|            5|        7.0|
+-------------+-----------+
only showing top 5 rows



So we can see that the `withColumn` method created a new column called `id_plus_two`, and set the values as the column `col("Randomized Id") + 2`.  

### Setting Dtypes

In [ ]:
updated_df.dtypes

[('Extract Run Date', 'string'),
 ('Randomized Id', 'string'),
 ('CCRB Received Year', 'string'),
 ('Days Between Incident Date and Received Date', 'string'),
 ('Case Type', 'string'),
 ('Complaint Received Place', 'string'),
 ('Complaint Received Mode', 'string'),
 ('Borough Of Incident', 'string'),
 ('Patrol Borough Of Incident', 'string'),
 ('Reason For Initial Contact', 'string'),
 ('id', 'int')]

Ok, so now let's use the `withColumn`, simply to change the values from `Randomized Id` from a string into an integer.  We can do so with the following:

In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

updated_df = complaints_df. \
            withColumn("id", col("Randomized Id").cast(IntegerType()))

In [ ]:
updated_df.dtypes

[('Extract Run Date', 'string'),
 ('Randomized Id', 'string'),
 ('CCRB Received Year', 'string'),
 ('Days Between Incident Date and Received Date', 'string'),
 ('Case Type', 'string'),
 ('Complaint Received Place', 'string'),
 ('Complaint Received Mode', 'string'),
 ('Borough Of Incident', 'string'),
 ('Patrol Borough Of Incident', 'string'),
 ('Reason For Initial Contact', 'string'),
 ('id', 'int')]

So we can see that the above statement took the values of `Randomized Id` and cast them into an integer.

Let's do this one more time changing the `Days Between Incident Date and Received Date` into a `Double`. 

> A double is a more precise version of a float (stores more decimal points).

In [ ]:
from pyspark.sql.types import DoubleType

new_df = updated_df.withColumn("days_between", col("Days Between Incident Date and Received Date").cast(DoubleType()))

And now if we look at the the datatypes, we can see that we now have a new column `days_between` of type `double`.

In [ ]:
new_df.dtypes

[('Extract Run Date', 'string'),
 ('Randomized Id', 'string'),
 ('CCRB Received Year', 'string'),
 ('Days Between Incident Date and Received Date', 'string'),
 ('Case Type', 'string'),
 ('Complaint Received Place', 'string'),
 ('Complaint Received Mode', 'string'),
 ('Borough Of Incident', 'string'),
 ('Patrol Borough Of Incident', 'string'),
 ('Reason For Initial Contact', 'string'),
 ('id', 'int'),
 ('days_between', 'double')]

In [ ]:
new_df.select("days_between").show(3)

+------------+
|days_between|
+------------+
|         2.0|
|        86.0|
|         0.0|
+------------+
only showing top 3 rows



### Changing the Column Name

Now one thing we may notice is that when using the `withColumn` function we do successfully add a new, properly coerced column.  But the old column is still hanging around.  

To correct that, there are a couple of things we can do.  One is to simply select the columns that are properly coerced.

In [ ]:
new_df.select(['id', 'days_between'])

DataFrame[id: int, days_between: double]

The other thing to do is to first change the column name, and then to coerce the data.  Let's see this. We'll start with our original dataframe.

In [ ]:
complaints_df.dtypes

[('Extract Run Date', 'string'),
 ('Randomized Id', 'string'),
 ('CCRB Received Year', 'string'),
 ('Days Between Incident Date and Received Date', 'string'),
 ('Case Type', 'string'),
 ('Complaint Received Place', 'string'),
 ('Complaint Received Mode', 'string'),
 ('Borough Of Incident', 'string'),
 ('Patrol Borough Of Incident', 'string'),
 ('Reason For Initial Contact', 'string')]

And then we'll change the column name.

In [ ]:
renamed_df = complaints_df.withColumnRenamed("Randomized Id", "id")

In [ ]:
renamed_df.dtypes

[('Extract Run Date', 'string'),
 ('id', 'string'),
 ('CCRB Received Year', 'string'),
 ('Days Between Incident Date and Received Date', 'string'),
 ('Case Type', 'string'),
 ('Complaint Received Place', 'string'),
 ('Complaint Received Mode', 'string'),
 ('Borough Of Incident', 'string'),
 ('Patrol Borough Of Incident', 'string'),
 ('Reason For Initial Contact', 'string')]

So we can see that this successfully renamed the column, but did not change the datatype.  For that we'll have to go back to the `withColumn` method, but this time with both the new and old column name being the same -- here id.  

Ok, here is both the `withColumnRenamed` method and the `withColumn` method.

In [ ]:
from pyspark.sql.types import IntegerType

updated_df = complaints_df.withColumnRenamed("Randomized Id", "id"). \
withColumn("id", col("id").cast(IntegerType()))

In [ ]:
updated_df.dtypes

[('Extract Run Date', 'string'),
 ('id', 'int'),
 ('CCRB Received Year', 'string'),
 ('Days Between Incident Date and Received Date', 'string'),
 ('Case Type', 'string'),
 ('Complaint Received Place', 'string'),
 ('Complaint Received Mode', 'string'),
 ('Borough Of Incident', 'string'),
 ('Patrol Borough Of Incident', 'string'),
 ('Reason For Initial Contact', 'string')]

So we can see that this time, we first renamed the `Randomized Id` column to be called `id`, and then we updated that `id` column's values to be of type integer.

### Changing Date Columns

Ok, so by now, we may be getting a sense of how to change our column datatypes.  We can begin by changing the name of the column with the `withColumnRenamed` function, and from there we can use the `withColumn` function to cast the data to a new datatype.

In [ ]:
from pyspark.sql.types import IntegerType

updated_df = complaints_df.withColumnRenamed("Randomized Id", "id"). \
withColumn("id", col("id").cast(IntegerType()))

Ok, one datatype that is a little trickier to coerce to is the datetype.  For example, let's try changing the `Extract Run Date` column similarly to how we did above. 

In [ ]:
from pyspark.sql.types import DateType, IntegerType
from pyspark.sql.functions import col

In [ ]:
updated_complaints_df = complaints_df. \
    withColumnRenamed("Extract Run Date", "complaint_date"). \
    withColumn("complaint_date", col("complaint_date").cast(DateType()))

In [ ]:
updated_complaints_df.select('complaint_date')

DataFrame[complaint_date: date]

But if we look at those values, we can see that they are all null.

In [ ]:
updated_complaints_df.select('complaint_date').show(2)

+--------------+
|complaint_date|
+--------------+
|          null|
|          null|
+--------------+
only showing top 2 rows



The issue is that our values did not start off in an easy to coerce format.  Let's take a look.

In [ ]:
complaints_df.select("Extract Run Date").show(2)

+----------------+
|Extract Run Date|
+----------------+
|      05/25/2018|
|      05/25/2018|
+----------------+
only showing top 2 rows



Ok, so to use the forward slashes, we'll need to do something like the following:

In [ ]:
from pyspark.sql.functions import col, to_date

updated_complaints_df = complaints_df. \
    withColumnRenamed("Extract Run Date", "complaint_date"). \
    withColumn("complaint_date", to_date("complaint_date", "MM/dd/yyyy"))

In [ ]:
updated_complaints_df.select("complaint_date").show(2)

+--------------+
|complaint_date|
+--------------+
|    2018-05-25|
|    2018-05-25|
+--------------+
only showing top 2 rows



Ok, so now we have properly formatted our date column.  And the benefit of coercing our data, is now we can use our SQL methods to query our data.  Let's see this quickly.

### Briefly Using SQL

The only thing we need to do is first set the name of our dataframe to a table name.

In [ ]:
updated_complaints_df.createOrReplaceTempView("complaints")

And from there, we are ready to go.

In [ ]:
spark.sql("SELECT complaint_date from complaints LIMIT 1").show()

+--------------+
|complaint_date|
+--------------+
|    2018-05-25|
+--------------+



And can use the date functions seen [here](https://dwgeek.com/spark-sql-date-and-timestamp-functions-and-examples.html/).

In [ ]:
spark.sql("""SELECT year(complaint_date)
    as year_of_complaint FROM complaints LIMIT 1""").show()

+-----------------+
|year_of_complaint|
+-----------------+
|             2018|
+-----------------+



### Summary

In this lesson, we saw how to coerce our data in to the proper dataypes.  We did so by learning about the `withColumn` method, which creates a new column derived from an original column.

### Resources

[Pyspark Operations](https://hendra-herviawan.github.io/)

[Spark SQL string Functions](https://sparkbyexamples.com/spark/usage-of-spark-sql-string-functions/)

[Pyspark From Pandas](https://databricks.com/session/data-wrangling-with-pyspark-for-data-scientists-who-know-pandas)